In [1]:
# nolint start: line_length_linter.
# I want to investigate this SS duplicate issue further.
# My fundamental worry is:
# 1. When loading in SS bills and cleaning them up, there were some duplicates that persisted.
# 2. They made their way into the bill details dataset.
# 3. These got joined with bill history, forming legis_data
# 4. And since legis_data is the df over which sles computation occurs, the analysis is corrupted.

# I'm going to investigate WI 23-24.
# My plan is to copy in the old estimate code that was used for treating the SS bils, and show how the duplciates persist through each stage.
# nolint end: line_length_linter.

In [2]:
# libs and consts
# Set repo root and load shared utilities
Sys.setenv(SLES_REPO_ROOT = normalizePath("."))
repo_root <- Sys.getenv("SLES_REPO_ROOT")

libs <- source(file.path(repo_root, "utils/libs.R"), local = TRUE)$value
paths <- source(file.path(repo_root, "utils/paths.R"), local = TRUE)$value
logging <- source(file.path(repo_root, "utils/logging.R"), local = TRUE)$value

# Extract functions
require_libs <- libs$require_libs
get_ss_dir <- paths$get_ss_dir
get_bill_dir <- paths$get_bill_dir
cli_log <- logging$cli_log
cli_warn <- logging$cli_warn

require_libs()

# consts
state <- "WI"
keep_types <- c("AB", "SB")
keep_types_regex <- paste0("^(", paste(keep_types, collapse = "|"), ")")
term <- "2023-2024"
term_years <- "2023_2024"
term_start_year <- "2023"
term_end_year <- "2024"

# paths using new utilities
ss_dir <- get_ss_dir(state)
bill_dir <- get_bill_dir(state)

ss_path_term_1 <- file.path(ss_dir, glue("{state}_SS_Bills_{term_start_year}.csv"))
ss_path_term_2 <- file.path(ss_dir, glue("{state}_SS_Bills_{term_end_year}.csv"))
bill_details_path <- file.path(bill_dir, glue("{state}_Bill_Details_{term_start_year}.csv"))
bill_hist_path <- file.path(bill_dir, glue("{state}_Bill_Histories_{term_start_year}.csv"))

print(ss_path_term_1)
print(ss_path_term_2)
print(bill_details_path)
print(bill_hist_path)

[1] "/home/nick/work/code/cel/sles/.data/WI/ss/WI_SS_Bills_2023.csv"
[1] "/home/nick/work/code/cel/sles/.data/WI/ss/WI_SS_Bills_2024.csv"
[1] "/home/nick/work/code/cel/sles/.data/WI/bill/WI_Bill_Details_2023.csv"
[1] "/home/nick/work/code/cel/sles/.data/WI/bill/WI_Bill_Histories_2023.csv"


In [3]:
# read and clean ss data
ss_bills <- bind_rows(read.csv(ss_path_term_1),
  read.csv(ss_path_term_2, colClasses = "character"),
)
# this is just copy paste from old WI - Estimate script
ss_bills <- ss_bills %>%
  filter(State == state) %>%  
  rename(bill_id = Bill.No) %>%
  mutate(
    Date = gsub("Sept", "Sep", Date),
    date = as.Date(gsub("\\.", "", Date), "%B %d, %Y"),
    year = as.integer(format(date, "%Y")),
    term = ifelse(year %% 2 == 1, paste0(year, "_", year + 1), paste0(year - 1, "_", year)),
    bill_id = toupper(bill_id),
    bill_id = gsub(" ", "", bill_id),
    bill_id = paste0(gsub("[0-9].+", "", bill_id), str_pad(gsub("^[A-Z]+", "", bill_id), 4, pad = "0")),
    SS = 1
  ) %>%
  select(state = State, term, year, bill_id, everything())

head(ss_bills)


,state,term,year,bill_id,Date,Title,Action,date,SS
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
1,WI,2023_2024,2023,AB0133,"Nov. 14, 2023",Amends Farmland Preservation Rules,Concurrence Vote Passed - Senate (31 - 2),2023-11-14,1
2,WI,2023_2024,2023,SB0312,"Nov. 14, 2023",Appropriates Funding for Grants to Local Governments and Landowners to Address PFAS Contamination,Bill Passed - Senate (22 - 11),2023-11-14,1
3,WI,2023_2024,2023,SB0166,"Nov. 9, 2023",Amends Punitive Articles for the Wisconsin Code of Military Justice,Concurrence Vote Passed - House (92 - 5),2023-11-09,1
4,WI,2023_2024,2023,SJR0078,"Nov. 9, 2023",Prohibits the Use of Private Funds to Pay for Election Equipment and Administration,Concurrence Vote Passed - House (60 - 35),2023-11-09,1
5,WI,2023_2024,2023,SJR0073,"Nov. 9, 2023",Amends the State Constitution to Add a Photo-ID Requirement to Vote,Concurrence Vote Passed - House (62 - 35),2023-11-09,1
6,WI,2023_2024,2023,SJR0071,"Nov. 9, 2023",Prohibits Non-Citizens from Voting in Any State Primary or General Election,Concurrence Vote Passed - House (60 - 34),2023-11-09,1


In [4]:
# read and clean bills data
bills <- read.csv(bill_details_path)
bill_history <- read.csv(bill_hist_path)

bills$term <- term_years
bills$session <- bills$session_type
bills <- select(bills, -session_type)

### Check for duplicates
if (nrow(bills) != nrow(distinct(bills))) {
  cat(" \n ~~~> DUPLICATE BILLS \n\n .")
  break
}

######## Standardize the Bill IDs
bills <- rename(bills, bill_id = bill_number) %>%
  mutate(bill_id = toupper(bill_id))

############### Drop Resolutions, Messages, Communications, Reports
bills <- mutate(bills, bill_type = toupper(gsub("[0-9].+", "", bill_id)))
all_bills <- bills
bills <- filter(bills, bill_type %in% keep_types) %>% select(-bill_type)

############### Standardize Sponsors
bills$LES_sponsor <- gsub("\\.$|\\. +$", "", tolower(bills$primary_sponsor))

### Split Bills with Multiple Introducing Sponsors
### *** NOTE: THese are NOT alphabetized so assuming order signifies primary introducer vs not
bills$LES_sponsor <- gsub(" and .+", "", bills$LES_sponsor)

######### Bills By Request
if (any(grepl("by request", bills$LES_sponsor))) {
  print("-----> BY REQUEST BILLS -- ADAPT SCRIPT --- BREAK")
  break
  # print(glue("-----> KEEPING {nrow(filter(bills, grepl('by request', LES_sponsor)))} bill(s) introduced BY REQUEST"))
  # bills$LES_sponsor <- str_trim(gsub('\\(by request\\)', '', bills$LES_sponsor))
}

#### Standardizing Accents in Names --- If left as is, will not match correctly to Klarner Data
bills$LES_sponsor <- gsub("á", "a", bills$LES_sponsor)
bills$LES_sponsor <- gsub("é", "e", bills$LES_sponsor)
bills$LES_sponsor <- gsub("ó", "o", bills$LES_sponsor)
bills$LES_sponsor <- gsub("í", "i", bills$LES_sponsor)

#### Name Fixes
if (any(bills$LES_sponsor == "molepske jr" | bills$LES_sponsor == "molepske jr.")) {
  bills[bills$LES_sponsor %in% c("molepske jr", "molepske jr."), ]$LES_sponsor <- "molepske"
}

# just print colnames and nrow 
print(colnames(bills))
print(nrow(bills))


 [1] "bill_id"             "session"             "primary_sponsor"    
 [4] "cosponsors"          "outchamber_sponsors" "status"             
 [7] "summary"             "bill_url"            "term"               
[10] "LES_sponsor"        
[1] 2343


In [5]:
# missingness check stage, precedes deduping
# First, we get the term-specific slice of the ss_bills df
ss_term <- ss_bills %>%
  filter(term == term_years) %>%
  distinct(term, bill_id, SS, year, Title) %>%
  mutate(SS = 1)

# then we're goiing to check to see if any bills in our ss df are missing from our bills df
# put anotehr way, if there are any bills in PVS that don't show up in the scraped data
missing_ss_bills <- anti_join(
  # x df is our pvs data
  ss_bills %>%
    # with an ss flag added
    mutate(SS = 1) %>%
    # and distinct on these columns
    distinct(term, bill_id, SS, year, Title),
  # y df is our scraped bills data
  bills,
  # join keys for the anti join
  by = c("bill_id", "term")
) %>%
  # after the anti-join, we're left with a df of bills that are in ss_bills but not in bills
  # we need to see if these are things we actually care about, though, or if they're
  # things we're going to filter out at a later stage anyway (e.g., resolutions, etc.)
    left_join(all_bills %>% select(bill_id, term, primary_sponsor), c("bill_id", "term")) %>%
    mutate(bill_type = toupper(gsub("[0-9].+|[0-9]+", "", bill_id))) %>%
    filter(bill_type %in% keep_types) %>%
    select(-bill_type) %>%
    filter(!grepl("committee", primary_sponsor, ignore.case = TRUE)) %>%
    arrange(primary_sponsor)

print(unique(missing_ss_bills$bill_id))

# this prints character(0), meaning after we filter out the stuff we don't care about, there's nothing kleft
# so all the bills in pvs data we care about, we have in our scraped bills data! great

character(0)


In [6]:
# next we go to dupe checking
# before this, I want to be clear about something:
# - There are duplicate bill ids in the ss_bills data
# - Among these duplicates, there are both exact and partial duplicates.
# - In partial duplicates, there are records with shared bill ids.
# - These bill ids do correspond to items of interest (i.e., %in% keep_types).
# - Some of these dupes persist into the ss_term df.
#   - This is because in the ss_term assignment, we make a distinct call on columns term, bill_id, SS, year, Title
#   - And, as we'll see below, there are records which share term, bill_id, SS, and TItle, but differ on year.

# Here are duplicated bill ids, ignoring all other attributes, filtered to items of interest.
ss_term_dupes <- ss_term %>%
  group_by(bill_id) %>%
  mutate(count = n()) %>%
  ungroup() %>%
  arrange(bill_id) %>%
  filter(count > 1) %>%
  select(bill_id, count) %>%
  filter(grepl("^(AB|SB)", bill_id)) %>% 
  distinct()

ss_bills_dupes <- ss_bills %>%
  group_by(bill_id) %>%
  mutate(count = n()) %>%
  ungroup() %>%
  arrange(bill_id) %>%
  filter(count > 1) %>%
  select(bill_id, count) %>%
  filter(grepl("^(AB|SB)", bill_id)) %>% 
  distinct()


ss_term_dupe_details <- ss_term %>%
  filter(bill_id %in% ss_term_dupes$bill_id) %>%
  select(bill_id, year, Title) %>%
  arrange(bill_id, desc(year))

ss_bills_dupe_details <- ss_bills %>%
  filter(bill_id %in% ss_bills_dupes$bill_id) %>%
  select(bill_id, year, Title, Date) %>%
  arrange(bill_id, desc(Date))



# So we have some exact duplicates, and then some partial duplicates.
# Most duplicates in the ss_bills df are distinct on Date. These don't make it into the ss_term df because if the aforementioned distinct call.
# But a few are also distinct on year. These do make it in.
print(head(ss_bills_dupe_details %>% mutate(Title = substr(Title, 1, 25))))
print(ss_term_dupe_details %>% mutate(Title = substr(Title, 1, 25))) 



  bill_id year                     Title           Date
1  AB0141 2023 Prohibits Local Restricti   June 7, 2023
2  AB0141 2023 Prohibits Local Restricti April 18, 2023
3  AB0142 2023 Prohibits Local Restricti   June 7, 2023
4  AB0142 2023 Prohibits Local Restricti April 18, 2023
5  AB0146 2023 Prohibits Local Governmen  Oct. 17, 2023
6  AB0146 2023 Prohibits Local Governmen April 25, 2023
   bill_id year                     Title
1   AB0377 2024 Prohibits Transgender Gir
2   AB0377 2023 Prohibits Transgender Gir
3   AB0415 2024 Establishes a 5-Member Ad
4   AB0415 2023 Establishes a 5-Member Ad
5   SB0139 2024 Establishes a Statewide W
6   SB0139 2023 Establishes a Statewide W
7   SB0145 2024 Increases Professional Au
8   SB0145 2023 Increases Professional Au
9   SB0312 2024 Appropriates Funding for 
10  SB0312 2023 Appropriates Funding for 


In [7]:
# Now that we've established what types of duplicates we have in our ss_term data, we're going to walk through the deduplication logic that exists in the script.
# This is, as far as I can tell, engineered to root out the issue in the above cell.
# Th way that de-duplication seems to work in the script is that we actually perform the joining (ss + bills) that we eventually want to do and then we see if that join produces duplicates.

# here is how the original script does it, in two phases:
# First dedupe phase
# take ss_term
duplicate_ss_bills <- ss_term %>%
  # insert a rowname col
  tibble::rownames_to_column() %>%
  # left join ss term <- bills on bill_id and term
  left_join(., bills, by = c("bill_id", "term")) %>%
  # Since each ss_term record had a unique id in the form of rowname before this join,
  # duplicates will be artifacted as multiple rows with the same rowname value
  group_by(rowname) %>%
  # count how many rows share each rowname
  mutate(count = n()) %>%
  # clean them up for viewing
  ungroup() %>%
  select(count, bill_id, term, session, Title, summary) %>%
  arrange(desc(count), bill_id)
  
# if there are any rows in this df, we have dupes
if (nrow(duplicate_ss_bills %>% filter(count > 1)) > 0) {
  print("you have duplicates")
  ss_duplicates_exist <- 1
  break
} else {
  print('no dupes')
  ss_duplicates_exist <- 0
}

# Getting the no dupes here means we bypass a manual review step, which I'll put here, commented out.
# if you have duplicated bills, you have to go into this if statement. otherwise, do the else logic.
# if (nrow(duplicate_SS_bills %>% filter(count > 1)) > 0) {
#   # now have to remove the duplicated bills that don't correspond
#   write.csv(duplicate_SS_bills, glue("../../../State Legislative Data/States/{this_state}/{this_state}_duplicate_SS_bills_{t_yrs}.csv"), row.names = F)
#   # edit this file in Excel, create a column called filter, put in the value "remove" if the Title from PVS doesn't match the bill description
#   SS_term <- read.csv(glue("../../../State Legislative Data/States/{this_state}/{this_state}_duplicate_SS_bills_{t_yrs}_edited.csv")) %>%
#     filter(filter != "remove") %>%
#     select(bill_id, term, session) %>%
#     mutate(SS = 1) %>%
#     distinct()
# } else {
# ...
# we'll get to this branch in the cells below, since this is the route our data actually takes.
# ...
# }

# Okay, and again, just to be clear, we still have some records that are identical on all fields except year in this df.
persisting_dupes <- ss_term %>%
  left_join(bills, by = c("bill_id", "term")) %>%
  group_by(bill_id, term) %>%  # Group by bill, not SS row
  mutate(count = n()) %>%
  filter(count > 1) %>% # Find bills matched by multiple SS entries
  # and let's filter out resolutions
  filter(grepl("^(AB|SB)", bill_id)) %>%
  arrange(bill_id, desc(year))

problem_bill_ids <- unique(persisting_dupes$bill_id)
print(persisting_dupes)

[1] "no dupes"
# A tibble: 10 × 14
# Groups:   bill_id, term [5]
   term      bill_id    SS  year Title        session primary_sponsor cosponsors
   <chr>     <chr>   <dbl> <int> <chr>        <chr>   <chr>           <chr>     
 1 2023_2024 AB0377      1  2024 Prohibits T… RS      Dittrich        Rettinger…
 2 2023_2024 AB0377      1  2023 Prohibits T… RS      Dittrich        Rettinger…
 3 2023_2024 AB0415      1  2024 Establishes… RS      Oldenburg       Novak; Tr…
 4 2023_2024 AB0415      1  2023 Establishes… RS      Oldenburg       Novak; Tr…
 5 2023_2024 SB0139      1  2024 Establishes… RS      Stafsholt       Cowles; F…
 6 2023_2024 SB0139      1  2023 Establishes… RS      Stafsholt       Cowles; F…
 7 2023_2024 SB0145      1  2024 Increases P… RS      Testin          Cabral-Gu…
 8 2023_2024 SB0145      1  2023 Increases P… RS      Testin          Cabral-Gu…
 9 2023_2024 SB0312      1  2024 Appropriate… RS      Wimberger       Cowles; B…
10 2023_2024 SB0312      1  2023 Appropriate

In [8]:
# second dedup phase
### Check Missing
print(colnames(bills))
ss_in_bills <- sum(bills$SS)
ss_in_PVS <- nrow(ss_term %>%
  mutate(bill_type = toupper(gsub("[0-9].+|[0-9]+", "", bill_id))) %>%
  filter(bill_type %in% keep_types))

if (ss_in_bills == ss_in_PVS) {
  print("All SS bills from PVS are in Bills data")
} else {
  print("MISSING SS BILLS IN BILLS DATA")
}


 [1] "bill_id"             "session"             "primary_sponsor"    
 [4] "cosponsors"          "outchamber_sponsors" "status"             
 [7] "summary"             "bill_url"            "term"               
[10] "LES_sponsor"        
[1] "MISSING SS BILLS IN BILLS DATA"


In [9]:
# I Guess the last question to answer is, if these bills are problematic, are they producing downstream issues in the sles estimation?
# so, what does the rest of the process look like

# As far as I can tell, it just ends here. That is, this is always going to fail for WI 23-24 because we're coming in with duplicates from the ss data.
orig_row_n <- c(nrow(bills), nrow(ss_term))
bills2 <- bills %>%
  left_join(ss_term %>% select(-Title) %>% distinct(), by = c("bill_id", "term")) %>%
  mutate(SS = ifelse(is.na(SS), 0, SS))
ss_term2 <- ss_term %>%
  left_join(bills %>% select(bill_id, term, session), by = c("bill_id", "term"))
if (!identical(c(nrow(bills2), nrow(ss_term2)), orig_row_n)) {
  print("merge failed")

  # difference should be 5, b/c we're coming in with those five problem bill ids..
  print(nrow(bills2) - nrow(bills))
  print('duplicate bill ids post-merge')
  bills2 %>%  
    filter(bill_id %in% problem_bill_ids) %>%
    arrange(bill_id) %>%
    select(bill_id) %>%
    group_by(bill_id) %>%
    distinct() %>%
    pull(bill_id) %>%
    print()
  print("problem bill ids that persisted through ss_term deduping:")
  print(problem_bill_ids)
} else {
  bills <- bills2
  ss_term <- ss_term2
  rm(bills2, ss_term2)
}

[1] "merge failed"
[1] 5
[1] "duplicate bill ids post-merge"
[1] "AB0377" "AB0415" "SB0139" "SB0145" "SB0312"
[1] "problem bill ids that persisted through ss_term deduping:"
[1] "AB0377" "AB0415" "SB0139" "SB0145" "SB0312"


In [12]:
# Alright, and that's where the original scripts fail. And there doesn't seem to be any documentation
# on how to resolve these kinds of errors, atleast nothing in the code. Maybe I've missed something?

# My preferred way to solve this is to improve the first pass of ss data deduping.

# Run this with true first, it produces diagnostic outputs that will show the same problematic bill ids
use_legacy_ss_dedup <- FALSE

if (use_legacy_ss_dedup) {
  # Legacy method: matches original implementation but allows year-duplicate SS bills
  # In my mind, I consider this "broken", but need to match original written code and produce runnable script.
  ss_term <- ss_bills %>%
    filter(term == term_years) %>%
    distinct(term, bill_id, SS, year, Title) %>%
    mutate(SS = 1) %>%
    mutate(bill_type = toupper(gsub("[0-9].+|[0-9]+", "", bill_id))) %>%
    filter(bill_type %in% keep_types) %>% 
    select(-bill_type)
} else {
  cli_log(glue('Deduping SS bills with use_legacy: {use_legacy_ss_dedup}', use_legacy_ss_dedup = use_legacy_ss_dedup))
  # This is how I want to resolve it.
  ss_term <- ss_bills %>%
    filter(term == term_years) %>%
    distinct(term, bill_id, SS, year, Title) %>%
    mutate(SS = 1) %>%
    mutate(bill_type = toupper(gsub("[0-9].+|[0-9]+", "", bill_id))) %>%
    filter(bill_type %in% keep_types) %>%
    select(-bill_type) %>%
    group_by(bill_id, term) %>%
    mutate(
      title_variations = n_distinct(Title),
      is_duplicate_years = n() > 1
    ) %>%
    ungroup() %>%
    filter(
      !is_duplicate_years |
        (is_duplicate_years & title_variations == 1)
    ) %>%
    # For identical titles, keep earliest year
    group_by(bill_id, term, Title) %>%
    slice_min(year, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(-title_variations, -is_duplicate_years)
}


if (use_legacy_ss_dedup) {
  ss_term_new <- ss_bills %>%
    filter(term == term_years) %>%
    distinct(term, bill_id, SS, year, Title) %>%
    mutate(SS = 1) %>%
    mutate(bill_type = toupper(gsub("[0-9].+|[0-9]+", "", bill_id))) %>%
    filter(bill_type %in% keep_types) %>%
    select(-bill_type) %>%
    group_by(bill_id, term) %>%
    mutate(title_variations = n_distinct(Title), is_duplicate_years = n() > 1) %>%
    ungroup() %>%
    filter(!is_duplicate_years | (is_duplicate_years & title_variations == 1)) %>%
    group_by(bill_id, term, Title) %>%
    slice_min(year, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(-title_variations, -is_duplicate_years)

  # Compare counts
  legacy_count <- nrow(ss_term)
  new_count <- nrow(ss_term_new)

  if (legacy_count != new_count) {
    cli_warn(glue("SS dedup difference: legacy={legacy_count} rows, improved={new_count} rows"))

    # which bills differ
    year_dupes <- ss_term %>%
      group_by(bill_id, term) %>%
      summarise(count = n(), years = paste(unique(year), collapse = ", "), .groups = "drop") %>%
      filter(count > 1)

    if (nrow(year_dupes) > 0) {
      cli_warn("Bills appearing in multiple years (legacy allows, improved deduplicates):")
      cli_warn_table(year_dupes)
    }
  } else {
    cli_log("No SS deduplication differences between methods for this term")
  }
  # rm(ss_term_new)
}

# and then we check like the original script does
duplicate_ss_bills <- 
  ss_term %>%
    # row numbers tas explicit col to help check for dupes
    tibble::rownames_to_column() %>%
    # join ss_term to bill data
    left_join(
      # x df is ss_term, coming in implicitly b/c we're in a pipe
      # y df
      bills,
      # join keys
      by = c("bill_id", "term")
    ) %>%
    # group by rownames that we added to ss term data that end up in bills after join
    group_by(rowname) %>%
    # sum rowname instances
    mutate(count = n()) %>%
    # ungroup but leave summary stats
    ungroup() %>%
    select(count, bill_id, term, session, Title, summary) %>%
    arrange(desc(count), bill_id)

filtered_dupes <- duplicate_ss_bills %>%
    filter(count > 1)

if (use_legacy_ss_dedup) {
  if (nrow(filtered_dupes) == 0) {
    cli_warn('Dupe detection found 0, but legacy mode true. If diagnostics revealed any dupes, consider re-running in non-legacy mode.')
  } else {
    cli_error_table(filtered_dupes)
  }
} else {
  if (nrow(filtered_dupes) == 0) {
    cli_log("No dupes found.")
  } else {
    cli_error_table(filtered_dupes)
  }
}


[L] [2026-01-02 19:22:30]: Deduping SS bills with use_legacy: FALSE

[L] [2026-01-02 19:22:30]: No dupes found.

